## 📊 Data Handling and Federated Partitioning

AG News dataset loading, preprocessing, and non-IID partitioning with drift injection capabilities.

In [ ]:
class BERTClassifier(nn.Module):
    \"\"\"BERT-tiny classifier optimized for federated learning.\"\"\"

    def __init__(self, model_name: str, num_classes: int = 4, dropout: float = 0.1):
        super().__init__()
        self.model_name = model_name
        self.num_classes = num_classes

        # Load BERT configuration and model
        self.config = AutoConfig.from_pretrained(model_name)
        self.bert = AutoModel.from_pretrained(model_name, config=self.config)

        # Classification head
        self.dropout = nn.Dropout(dropout)
        self.classifier = nn.Linear(self.config.hidden_size, num_classes)

        # Initialize classifier weights
        nn.init.normal_(self.classifier.weight, std=0.02)
        nn.init.zeros_(self.classifier.bias)

    def forward(self, input_ids, attention_mask, labels=None):
        # BERT forward pass
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)

        # Use [CLS] token representation
        pooled_output = outputs.pooler_output
        pooled_output = self.dropout(pooled_output)

        # Classification
        logits = self.classifier(pooled_output)

        loss = None
        if labels is not None:
            loss_fn = nn.CrossEntropyLoss()
            loss = loss_fn(logits, labels)

        return {'loss': loss, 'logits': logits, 'hidden_states': outputs.last_hidden_state}

    def get_embeddings(self, input_ids, attention_mask):
        \"\"\"Extract embeddings for drift detection.\"\"\"
        with torch.no_grad():
            outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
            # Return [CLS] token embeddings
            return outputs.pooler_output


def create_model_and_tokenizer(config: Dict[str, Any], device: torch.device):
    \"\"\"Create BERT model and tokenizer.\"\"\"
    model_name = config['model']['model_name']
    num_classes = config['model']['num_classes']
    dropout = config['model']['dropout']

    # Create tokenizer
    tokenizer = AutoTokenizer.from_pretrained(model_name)

    # Create model
    model = BERTClassifier(model_name, num_classes, dropout)
    model = model.to(device)

    # Enable mixed precision if GPU available
    if device.type == 'cuda':
        try:
            model = model.half()  # Use FP16 for memory efficiency
            print(\"🚀 FP16 mixed precision enabled\")\n        except:\n            print(\"⚠️ FP16 not supported, using FP32\")\n\n    return model, tokenizer\n\n\nprint(\"✅ BERT model implementation ready!\")"

## 🤖 BERT Model Implementation

BERT-tiny classifier with GPU optimization for federated learning.

# 🔄 Federated LLM Drift Detection and Recovery System

**A comprehensive standalone implementation for Google Colab with GPU acceleration**

## 📋 System Overview

This notebook implements a sophisticated **multi-level drift detection architecture** for federated learning with BERT-tiny models:

### 🏗️ **Architecture Components**
- **Client-Side Detection**: ADWIN (concept drift) + Evidently (data drift)
- **Server-Side Detection**: MMD statistical test on embedding aggregates  
- **Adaptive Mitigation**: FedAvg → FedTrimmedAvg when drift detected
- **Synthetic Drift Injection**: Vocabulary shift, label noise, distribution shift

### 🎯 **Key Features**
- GPU-optimized for Google Colab (T4/P100/V100)
- Real-time drift monitoring with visual analytics
- Configurable drift scenarios and client heterogeneity
- Production-ready federated learning pipeline

---

## 🚀 Environment Setup and Dependencies

**Important**: Restart runtime after running this cell to avoid dependency conflicts.

In [ ]:
# Fix dependency conflicts and install packages
import subprocess
import sys

def install_package(package):
    try:
        subprocess.check_call([sys.executable, "-m", "pip", "install", "--no-deps", package])
        print(f"✅ {package} installed successfully")
    except Exception as e:
        print(f"⚠️ {package} installation issue: {e}")

# Core ML packages (install without dependencies to avoid conflicts)
print("📦 Installing core packages...")
!pip install -q --upgrade pip

# Install PyTorch with CUDA support
!pip install -q torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

# Install essential packages
!pip install -q transformers>=4.30.0
!pip install -q datasets>=2.0.0
!pip install -q scikit-learn>=1.0.0
!pip install -q pandas>=1.3.0
!pip install -q numpy>=1.21.0
!pip install -q matplotlib seaborn plotly

# Install Flower for federated learning
!pip install -q flwr>=1.0.0

# Install drift detection packages (with fallbacks)
try:
    !pip install -q river>=0.20.0
    print("✅ River (ADWIN) installed")
except:
    print("⚠️ River installation failed - will use fallback drift detection")

try:
    !pip install -q alibi-detect>=0.11.0
    print("✅ Alibi-detect (MMD) installed")
except:
    print("⚠️ Alibi-detect installation failed - will use fallback MMD")

try:
    !pip install -q evidently>=0.4.0
    print("✅ Evidently installed")
except:
    print("⚠️ Evidently installation failed - will use fallback data drift detection")

try:
    !pip install -q nlpaug>=1.1.0
    print("✅ NLPAug installed")
except:
    print("⚠️ NLPAug installation failed - will use simplified drift injection")

print("\n🔄 **IMPORTANT**: Please restart the runtime now (Runtime → Restart runtime)")
print("Then run the next cell to continue with the setup.")

In [ ]:
# Configure GPU and environment (Run after restart)
import torch
import os
import gc
import warnings
warnings.filterwarnings('ignore')

# Set environment variables for optimal performance
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'
os.environ['TOKENIZERS_PARALLELISM'] = 'false'

# Check GPU availability and configure
def setup_device():
    if torch.cuda.is_available():
        device = torch.device('cuda')
        print(f"🎮 GPU Available: {torch.cuda.get_device_name(0)}")
        print(f"📊 GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")
        
        # Optimize memory usage
        torch.cuda.empty_cache()
        torch.backends.cudnn.benchmark = True
        
        # Set memory fraction to prevent OOM
        if hasattr(torch.cuda, 'set_per_process_memory_fraction'):
            torch.cuda.set_per_process_memory_fraction(0.8)
            
    else:
        device = torch.device('cpu')
        print("⚠️ No GPU available, using CPU")
    
    return device

device = setup_device()
print(f"🔧 Device configured: {device}")

In [ ]:
# Import all required libraries with fallback handling
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import json
import logging
import time
import random
from datetime import datetime
from pathlib import Path
from typing import Dict, List, Tuple, Optional, Any, Callable
from collections import defaultdict

# ML and DL imports
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModel, AutoConfig
from sklearn.metrics import accuracy_score, classification_report
from datasets import load_dataset

# Federated Learning
import flwr as fl
from flwr.simulation import start_simulation
from flwr.common import Context, Parameters, Scalar
from flwr.server.strategy import FedAvg

# Drift Detection imports with fallbacks
DRIFT_PACKAGES = {
    'river': False,
    'alibi_detect': False,
    'evidently': False,
    'nlpaug': False
}

try:
    from river.drift import ADWIN
    DRIFT_PACKAGES['river'] = True
    print("✅ River (ADWIN) imported")
except ImportError:
    print("⚠️ River not available - using fallback ADWIN")
    
try:
    from alibi_detect import MMDDrift
    DRIFT_PACKAGES['alibi_detect'] = True
    print("✅ Alibi-detect (MMD) imported")
except ImportError:
    print("⚠️ Alibi-detect not available - using fallback MMD")

try:
    from evidently.report import Report
    from evidently.metric_preset import DataDriftPreset
    DRIFT_PACKAGES['evidently'] = True
    print("✅ Evidently imported")
except ImportError:
    print("⚠️ Evidently not available - using fallback data drift detection")

try:
    import nlpaug.augmenter.word as naw
    DRIFT_PACKAGES['nlpaug'] = True
    print("✅ NLPAug imported")
except ImportError:
    print("⚠️ NLPAug not available - using simplified drift injection")

# Set up logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

print(f"\n✅ All imports loaded successfully!")
print(f"📦 Available packages: {[k for k, v in DRIFT_PACKAGES.items() if v]}")

## ⚙️ Configuration and Constants

Define all configuration parameters for the federated learning simulation.

In [ ]:
# Global Configuration for Federated Learning Drift Detection System
CONFIG = {
    # Model configuration
    'model': {
        'model_name': 'prajjwal1/bert-tiny',
        'num_classes': 4,
        'max_length': 128,
        'batch_size': 16,
        'learning_rate': 2e-5,
        'num_epochs': 3,
        'warmup_steps': 100,
        'dropout': 0.1
    },

    # Federated learning configuration
    'federated': {
        'num_clients': 10,
        'alpha': 0.5,  # Dirichlet concentration for non-IID
        'min_samples_per_client': 50
    },

    # Drift configuration
    'drift': {
        'injection_round': 25,
        'drift_intensity': 0.3,
        'affected_clients': [2, 5, 8],  # Which clients get drift
        'drift_types': ['label_noise', 'vocab_shift']
    },

    # Drift detection configuration
    'drift_detection': {
        'adwin_delta': 0.002,
        'mmd_p_val': 0.05,
        'mmd_permutations': 100,
        'evidently_threshold': 0.25,
        'trimmed_beta': 0.2,  # For FedTrimmedAvg
    },

    # Simulation configuration
    'simulation': {
        'num_rounds': 50,
        'fraction_fit': 1.0,
        'fraction_evaluate': 1.0,
        'min_fit_clients': 2,
        'min_evaluate_clients': 2,
        'mitigation_threshold': 0.3  # >30% clients reporting drift
    }
}

print("✅ Configuration loaded:")
print(f"📊 Clients: {CONFIG['federated']['num_clients']}")
print(f"🔄 Rounds: {CONFIG['simulation']['num_rounds']}")
print(f"💥 Drift injection: Round {CONFIG['drift']['injection_round']}")
print(f"🎯 Affected clients: {CONFIG['drift']['affected_clients']}")